In [25]:
import numpy as np
import pandas as pd
import nltk
import hazm
from hazm import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('per.csv', encoding='utf8')

In [3]:
data.head(10)

,NewsID,Title,Body,Date,Time,Category,Category2
0,843656,\nوزير علوم درجمع استادان نمونه: سن بازنشستگي ...,\nوزير علوم در جمع استادان نمونه كشور گفت: از ...,\n138/5//09,\n0:9::18,\nآموزشي-,\nآموزشي
1,837144,\nگردهمايي دانش‌آموختگان موسسه آموزش عالي سوره...,\nبه گزارش سرويس صنفي آموزشي خبرگزاري دانشجويا...,\n138/5//09,\n1:4::11,\nآموزشي-,\nآموزشي
2,436862,\nنتايج آزمون دوره‌هاي فراگير دانشگاه پيام‌نور...,\nنتايج آزمون دوره‌هاي فراگير مقاطع كارشناسي و...,\n138/3//07,\n1:0::03,\nآموزشي-,\nآموزشي
3,227781,\nهمايش يكروزه آسيب شناسي مفهوم روابط عمومي در...,\n,\n138/2//02,\n1:3::42,\nاجتماعي-خانواده-,\nاجتماعي
4,174187,\nوضعيت اقتصادي و ميزان تحصيلات والدين از مهمت...,\nمحمدتقي علوي يزدي، مجري اين طرح پژوهشي در اي...,\n138/1//08,\n1:1::49,\nآموزشي-,\nآموزشي
5,768531,\nدانشكده علوم حديث همزمان با سال پيامبر اعظم ...,\nدوره آموزشي سيره نبوي بعنوان يكي از رشته‌هاي...,\n138/5//05,\n1:1::18,\nآموزشي-,\nآموزشي
6,395642,\nنخستين مركز آموزش عالي بين‌المللي دانشگاه مو...,\nبه گزارش خبرنگار اعزامي خبرگزاري دانشجويان ا...,\n138/3//03,\n1:1::52,\nآموزشي-,\nآموزشي
7,176676,\nدومين گردهمايي انجمن‌هاي علمي دانشجويي دانشگ...,\nبه گزارش سرويس آموزشي ايسنا، دفتر انجمن‌هاي ...,\n138/1//09,\n0:9::44,\nآموزشي-,\nآموزشي
8,787861,\nتقويم تحصيلي دانشگاه پيام نور مشهد اعلام شد,\nبه گزارش خبرگزاري دانشجويان ايران (ايسنا) - ...,\n138/5//06,\n0:8::38,\nآموزشي-,\nآموزشي
9,142104,\nهفتمين اجلاس مشترك هياتهاي امناي دانشگاهها و...,\nبه گزارش خبرنگار آموزشي ايسنا، در اين اجلاس ...,\n138/1//04,\n1:1::58,\nآموزشي-,\nآموزشي


In [5]:
with open('stopwords.txt', encoding='utf8') as stopwords_file:
    stopwords = stopwords_file.readlines()
stopwords = [line.replace('\n', '') for line in stopwords]
stopwords

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '...',
 '....',
 '/',
 ':',
 '<',
 '=',
 '>',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~',
 '«',
 '»',
 '،',
 '؛',
 '؟',
 'ء',
 'آاو و و و',
 'آباد',
 'آخ',
 'آخر',
 'آخرها',
 'آخه',
 'آدمهاست',
 'آرام',
 'آرام آرام',
 'آره',
 'آری',
 'آزادانه',
 'آسان',
 'آسانی',
 'آسيب پذيرند',
 'آسیب',
 'آسیب پذیرند',
 'آشكارا',
 'آشنايند',
 'آشنایند',
 'آشکارا',
 'آقا',
 'آقای',
 'آقایان',
 'آمد',
 'آمدن',
 'آمده',
 'آمرانه',
 'آن',
 'آن ها',
 'آن گاه',
 'آنان',
 'آناني',
 'آنانی',
 'آنجا',
 'آنرا',
 'آنطور',
 'آنقدر',
 'آنكه',
 'آنها',
 'آنهاست',
 'آنچنان',
 'آنچنان كه',
 'آنچنان که',
 'آنچه',
 'آنکه',
 'آنگاه',
 'آن\u200cها',
 'آهان',
 'آهاي',
 'آهای',
 'آور',
 'آورد',
 'آوردن',
 'آورده',
 'آوه',
 'آيا',
 'آيد',
 'آی',
 'آیا',
 'آید',
 'آیند',
 'ا',
 'ابدا',
 'ابلهانه',
 'ات',
 'اتفاقا',
 'اثر',
 'اثرِ',
 'اجراست',
 'احتراما',
 'احتمالا',
 'احياناً',
 'احیانا',
 'احیاناً',
 'اخ',
 'اختصارا',

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/masoud533/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
nltk_stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(nltk_stopwords)
len(stopwords)

2976

In [12]:
stemmer = hazm.Stemmer()

In [13]:
stemmer.stem('کتاب ها')

'کتاب '

In [17]:
dataset = pd.DataFrame(columns=('title_body', 'category'))
for index, row in data.iterrows():
    title_body = row['Title'] + ' ' + row['Body']
    title_body_tokenized = word_tokenize(title_body)
    title_body_tokenized_filtered = [w for w in title_body_tokenized if not w in stopwords]
    title_body_tokenized_filtered_stemmed = [stemmer.stem(w) for w in title_body_tokenized_filtered]
    dataset.loc[index] = {
        'title_body': ' '.join(title_body_tokenized_filtered_stemmed),
        'category': row['Category2'].replace('\n', '')
    }

In [18]:
dataset.head(10)

,title_body,category
0,وزير علو درجمع استاد نمونه سن بازنشستگي استاد ...,آموزشي
1,گردهمايي دانش‌آموختگ موسسه آموز عالي سوره برگز...,آموزشي
2,نتايج آزمون دوره‌هاي فراگير دانشگاه پيام‌نور ن...,آموزشي
3,هماي يكروزه آسيب مفهو روابط عمومي بابلسر برگزا...,اجتماعي
4,وضعي اقتصادي تحصيل والدين مهمترين عوامل موفقي ...,آموزشي
5,دانشكده علو حديث پيامبر اعظ برگزار مي‌كند دوره...,آموزشي
6,مركز آموز عالي بين‌المللي دانشگاه مونا استرالي...,آموزشي
7,دومين گردهمايي انجمن‌هاي علمي دانشجويي دانشگاه...,آموزشي
8,تقوي تحصيلي دانشگاه پيا نور مشهد گزار خبرگزاري...,آموزشي
9,هفتمين اجلاس مشترك هياتهاي امناي دانشگاه موسس ...,آموزشي


In [20]:
vectorizer = TfidfVectorizer()
vectorizer.fit(dataset['title_body'])

TfidfVectorizer()

In [22]:
# import pickle
# with open('Tfidf.model', 'wb') as file:
#     pickle.dump(vectorizer, file)

In [23]:
X = vectorizer.transform(dataset['title_body'])

In [24]:
X

<10999x60631 sparse matrix of type '<class 'numpy.float64'>'
	with 1556652 stored elements in Compressed Sparse Row format>

In [26]:
le = LabelEncoder()
y = le.fit_transform(dataset['category'])

In [27]:
le.classes_

array(['آموزشي', 'اجتماعي', 'اقتصادي', 'بهداشتي', 'تاريخي', 'سياسي',
       'علمي', 'فرهنگي', 'فقه و حقوق', 'مذهبي', 'ورزشي'], dtype=object)